In [1]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('../../../data/raw/river/arno.csv')

# some of our features are missing loads of datapoints;
# the thinking here is: if the amount of rainfall at a
# certain location hasn't been measured in 13 years,
# are we to expect it will suddenly come into play for
# new predictions? drop these "legacy" features
df = df.drop([
    'Rainfall_Vernio', 'Rainfall_Stia', 'Rainfall_Consuma', 'Rainfall_Incisa',
    'Rainfall_Montevarchi', 'Rainfall_S_Savino', 'Rainfall_Laterina',
    'Rainfall_Bibbiena', 'Rainfall_Camaldoli'
], axis=1)

# dates are in format DD/MM/YYYY
# convert to this explicitly
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# we have temperature data going back to 2000,
# but none of the rainfall data starts until 2004
df = df[df['Date'] > datetime(2003, 12, 31)]

# temperature data only goes through 2017
df = df[df['Date'] < datetime(2017, 3, 9)]

# a six month block is missing target data;
# drop these rows as well
df = df.drop(index=range(3835,4018))

# there are still a few 0s in the target that
# should be null; nullify them
df['Hydrometry_Nave_di_Rosano'] = df['Hydrometry_Nave_di_Rosano'].replace(0, float('nan'))

In [3]:
# There are still some missing values in our data that
# look approximately like this:
#
# 1/1/2000 - 1.9
# 1/2/2000 - 1.7
# 1/3/2000 - NaN
# 1/4/2000 - NaN
# 1/5/2000 - 2.3
# 1/6/2000 - 2.2
#
# The idea here is to interpolate any missing values
# based on the nearest non-null values; so the above
# would be imputed "by steps" like so:
#
# 1/2/2000 - 1.7
# 1/3/2000 - 1.9 <- imputed
# 1/4/2000 - 2.1 <- imputed
# 1/5/2000 - 2.3

def find_previous_nonnull(df, ix, col):
    i = ix
    while df.loc[i][col] != df.loc[i][col]:
        i -= 1
    return i

def find_next_nonnull(df, ix, col):
    i = ix
    while df.loc[i][col] != df.loc[i][col]:
        i += 1
    return i

def impute_at(df, ix, col):
    a = find_previous_nonnull(df, ix, col)
    b = find_next_nonnull(df, ix, col)
    steps = b - a
    dist = df.loc[b][col] - df.loc[a][col]
    off = ix - a
    df.at[ix, col] = df.loc[a][col] + off * dist / steps

def impute_by_step(df, col):
    ix = df[df[col].isnull()].index
    for i in ix:
        impute_at(df, i, col)

impute_by_step(df, 'Hydrometry_Nave_di_Rosano')
impute_by_step(df, 'Temperature_Firenze')

In [4]:
# center & standardize

# we have to picky about the date, which is an invalid data type
sansdate = df.drop('Date', axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(sansdate.values)
df2 = pd.DataFrame(X, columns=sansdate.columns)

# add the dates back in
df2['Date'] = df['Date'].reset_index(drop=True)

In [5]:
# because our data contains a large schism, which may not
# be easy to discern later, we'll just save it as two different datasets

dfA = df2[:1643]
dfB = df2[1644:]

dfA.to_csv('../../../data/clean/river/arnoA.csv', index=False)
dfB.to_csv('../../../data/clean/river/arnoB.csv', index=False)